In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data excel file 
df = pd.read_excel('Example_Data.xlsx',sheet_name='Example_Data',header=1)
df.rename(columns={'Company ID':'Company_ID','Company Name':'Company_Name','Fiscal Year':'Fiscal_Year','SIC Code':'SIC_Code','Trading Currency':'Trading_Currency'}, inplace=True)
df.head()

,Company_ID,Company_Name,Fiscal_Year,Industry,SIC_Code,Trading_Currency,SP,CDS,APD,ARD,ADA
0,4993687,A Company,2010,Machinery,6085,USD,75,564.0,109,44,0
1,4993687,A Company,2011,Machinery,6085,USD,66,721.0,102,47,0
2,4993687,A Company,2012,Machinery,6085,USD,29,1048.0,97,45,0
3,4993687,A Company,2013,Machinery,6085,USD,56,914.0,96,40,0
4,4993687,A Company,2014,Machinery,6085,3$,55,950.0,91,34,0


In [3]:
df.describe()

,Company_ID,SP,CDS,APD,ARD,ADA
count,1.110000e+02,111.000000,59.000000,111.000000,111.000000,111.000000
mean,4.269428e+06,92.144144,293.305085,86.900901,80.837838,0.234234
std,1.666143e+06,65.770655,292.946697,40.086034,41.616112,0.571361
min,4.988560e+05,0.000000,51.000000,37.000000,22.000000,0.000000
25%,4.976728e+06,44.500000,103.000000,65.000000,66.500000,0.000000
50%,4.991368e+06,68.000000,149.000000,84.000000,81.000000,0.000000
75%,4.994275e+06,123.000000,349.500000,98.000000,91.000000,0.000000
max,5.081187e+06,277.000000,1048.000000,396.000000,397.000000,2.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company_ID        111 non-null    int64  
 1   Company_Name      111 non-null    object 
 2   Fiscal_Year       111 non-null    object 
 3   Industry          111 non-null    object 
 4   SIC_Code          111 non-null    object 
 5   Trading_Currency  111 non-null    object 
 6   SP                111 non-null    int64  
 7   CDS               59 non-null     float64
 8   APD               111 non-null    int64  
 9   ARD               111 non-null    int64  
 10  ADA               111 non-null    int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 9.7+ KB


### Data Cleaning and Data Processing

In [5]:
# Delete duplicate data
df.drop_duplicates(inplace=True)

In [6]:
## Method to get index of invalid data 
# Get unique company_id
def get_uniqueCompanyID_index(company_id):
    uniCompanyID_index = []
    
    for i in range(0,len(company_id)):
        uniCompanyID_index.append(df[df['Company_ID']==company_id[i]].index.tolist())
    return uniCompanyID_index

# Fiscal year
def get_fiscalYear_index(values,year_list):
    fy_index = []
    for year in values:
         if year not in year_list:
                fy_index.append(df[df['Fiscal_Year']==year].index.tolist())
    return fy_index

# Sic Code
def get_siccode_index(values):
    value_ls = []
    for value in values:
        if len(str(value)) != 4:
            value_ls.append(value)
        elif not str(value).isdigit():
            value_ls.append(value)
            
    sc_index = []
    for i in range(0,len(value_ls)):
        sc_index.append(df[df['SIC_Code']==value_ls[i]].index.tolist())
    return sc_index

# Trading Currency
def get_currency_index(values,currency_list):
    tc_index = []
    for value in values:
         if value not in currency_list:
                tc_index.append(df[df['Trading_Currency']==value].index.tolist())
    return tc_index

## Delete method to delete invalid data with index
def delete_index(index):
    index = np.array(index).flatten()
    for i in range(0,len(index)):
        df.drop(index[i],inplace=True)


- Clean data that Company ID and Company Name not corresponding

In [7]:
# filter those ID's that appear with multiple names
non_unique = df.groupby('Company_ID').Company_Name.transform('nunique').ne(1)
df_unique = df[non_unique]
unique_id = df_unique.Company_ID.unique()

# get index of those's ID that appear with multiple names
unique_index = get_uniqueCompanyID_index(unique_id)

# delete data
delete_index(unique_index)

In [8]:
# df_unique # to check result

- Fiscal Year

In [9]:
# Fiscal Year
df['Fiscal_Year'].fillna(0)

f_year = [i for i in range(1999,2022)]
fy_values = df['Fiscal_Year'].values

fy_index = get_fiscalYear_index(fy_values,f_year) # call function to get index
delete_index(fy_index) # call function to delete invalid sic code data


In [10]:
df.iloc[fy_index[0][0]] # check delete function work or not

Company_ID            4994275
Company_Name        D Company
Fiscal_Year              2012
Industry            Machinery
SIC_Code                 5722
Trading_Currency          USD
SP                        120
CDS                       NaN
APD                        68
ARD                        77
ADA                         0
Name: 31, dtype: object

- SIC Code

In [11]:
# SIC code
siccode_value = df['SIC_Code'].values
sc_index = get_siccode_index(siccode_value) # call function to get index
delete_index(sc_index) # call function to delete invalid sic code data

In [12]:
df.iloc[sc_index[0][0]] # check delete function work or not

Company_ID            5081187
Company_Name        M Company
Fiscal_Year              2011
Industry            Machinery
SIC_Code                 5886
Trading_Currency          GBP
SP                         31
CDS                       NaN
APD                        87
ARD                       123
ADA                         2
Name: 103, dtype: object

- Trading currency

In [13]:
# Trading currency
# Trading currency only should be USD,GBP
currency_list = ['USD','GBP']
tc_values = df['Trading_Currency'].values

tc_index = get_currency_index(tc_values,currency_list) # call function to get index
delete_index(tc_index) # call function to delete invalid sic code data

In [14]:
df.iloc[tc_index[0][0]] # check delete function work or not

Company_ID            4993687
Company_Name        A Company
Fiscal_Year              2015
Industry            Machinery
SIC_Code                 6085
Trading_Currency          USD
SP                         19
CDS                       956
APD                        96
ARD                        28
ADA                         0
Name: 5, dtype: object

In [15]:
# Transform Metric info data type to integer number
df[['SP','CDS','APD','ARD','ADA']] = df[['SP','CDS','APD','ARD','ADA']].fillna(0).astype(int)

### Data Comparison

In [16]:
df.to_csv('./New_Example_Data',index=False, header=True)
df_DATA = pd.read_csv('./New_Example_Data')

In [17]:
df_DATA.head()

,Company_ID,Company_Name,Fiscal_Year,Industry,SIC_Code,Trading_Currency,SP,CDS,APD,ARD,ADA
0,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0
1,4993687,A Company,2011,Machinery,6085,USD,66,721,102,47,0
2,4993687,A Company,2012,Machinery,6085,USD,29,1048,97,45,0
3,4993687,A Company,2013,Machinery,6085,USD,56,914,96,40,0
4,4993687,A Company,2015,Machinery,6085,USD,19,956,96,28,0


In [18]:
# Read Example_DB data
df_DB = pd.read_excel('Example_Data.xlsx',sheet_name='Example_DB',header=0)
df_DB.rename(columns={'Company ID':'Company_ID','Company Name':'Company_Name','Fiscal Year':'Fiscal_Year','SIC Code':'SIC_Code','Trading Currency':'Trading_Currency','Metric Name':'Metric_Name'}, inplace=True)
df_DB.head()

,Company_ID,Company_Name,Fiscal_Year,Industry,SIC_Code,Trading_Currency,Metric_Name,Value
0,4993687,A Company,2010,Machinery,6085,USD,SP,75.0
1,4993687,A Company,2011,Machinery,6085,USD,SP,66.0
2,4993687,A Company,2012,Machinery,6085,USD,SP,29.0
3,4993687,A Company,2013,Machinery,6085,USD,SP,56.0
4,4993687,A Company,2015,Machinery,6085,USD,SP,19.0


In [19]:
df_DB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company_ID        429 non-null    int64  
 1   Company_Name      429 non-null    object 
 2   Fiscal_Year       429 non-null    int64  
 3   Industry          429 non-null    object 
 4   SIC_Code          429 non-null    int64  
 5   Trading_Currency  429 non-null    object 
 6   Metric_Name       429 non-null    object 
 7   Value             381 non-null    float64
dtypes: float64(1), int64(3), object(4)
memory usage: 26.9+ KB


In [20]:
# merge
df = pd.merge(df_DATA,df_DB,how='inner',sort=False,left_index=False,right_index=False)
df

,Company_ID,Company_Name,Fiscal_Year,Industry,SIC_Code,Trading_Currency,SP,CDS,APD,ARD,ADA,Metric_Name,Value
0,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,SP,75.0
1,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,CDS,564.0
2,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,APD,109.0
3,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,ARD,44.0
4,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,ADA,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,5081187,M Company,2018,Machinery,5886,GBP,97,0,104,121,1,SP,97.0
425,5081187,M Company,2018,Machinery,5886,GBP,97,0,104,121,1,CDS,NaN
426,5081187,M Company,2018,Machinery,5886,GBP,97,0,104,121,1,APD,104.0
427,5081187,M Company,2018,Machinery,5886,GBP,97,0,104,121,1,ARD,121.0


In [21]:
df_comparison = df[df.isnull().T.any()]

In [22]:
df_comparison = df_comparison[['Company_ID','Company_Name','Fiscal_Year','Industry','SIC_Code','Trading_Currency','Metric_Name']]
df_comparison['Data_in_DB'] = ' '
df_comparison['Data_in_File'] = ' '
df_comparison['Error_Type'] = 'Not in DB'
df_comparison

,Company_ID,Company_Name,Fiscal_Year,Industry,SIC_Code,Trading_Currency,Metric_Name,Data_in_DB,Data_in_File,Error_Type
31,4993687,A Company,2017,Machinery,6085,USD,CDS,,,Not in DB
86,4994275,D Company,2010,Machinery,5722,USD,CDS,,,Not in DB
101,4994275,D Company,2013,Machinery,5722,USD,CDS,,,Not in DB
106,4994275,D Company,2014,Machinery,5722,USD,CDS,,,Not in DB
111,4994275,D Company,2015,Machinery,5722,USD,CDS,,,Not in DB
116,4994275,D Company,2016,Machinery,5722,USD,CDS,,,Not in DB
121,4994275,D Company,2017,Machinery,5722,USD,CDS,,,Not in DB
126,4994275,D Company,2018,Machinery,5722,USD,CDS,,,Not in DB
131,4994275,D Company,2019,Machinery,5722,USD,CDS,,,Not in DB
181,4994208,F Company,2009,Machinery,6085,USD,CDS,,,Not in DB
